## Furniture Company -  Version 1

This is the first version of the furniture company problem.
All values are arbitrary and will get examined in this notebook.

This example generates a linear problem to simulate a year of the furniture company.
After that the optimal solution will be reviewed and Agents will solve the problem.

The notebook can be used as a template for different combinations of a model.


### Problem

+ Each chair requires
    + 1 wood, 1 metal
    + 2 hours of production time in A and 1 hour in B
+ Each desk requires
    + 2 wood, 1 metal
    + 3 hours of production time in A and 1 hour in B

+ Each bed requires 
    + 2 wood, 1 metal
    + 1 hour of production time in C and 2 hours in D

+ Each bookcase requires
    + 3 wood, 1 metal
    + 3 hours of production time in C and 1 hour in D

The profit of each product changes over the year, have a look at the profit variables
    
There are four workshops available, workshop A, B, C and D.
 
Available hours per month are:
+ 120 for A
+ 50 for B
+ 150 for C
+ 100 for D

Available materials change per month.




## Model

In [ ]:
# This is used to import the modules from the experiment folder
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
# First the model builder functionality needs to be imported.
from model_builder import Item, ModelBuilder

In [ ]:
# These values are needed to build the model

# define the profit per month for each item:

chair_profit = [3,3,2,3,3,2,3,2,3,2,3,3]
table_profit = [8,10,8,9,10,9,8,9,10,7,7,10]
bed_profit = [11,12,11,13,9,8,13,11,11,13,13,12]
bookcase_profit = [20,21,19,20,20,24,20,19,15,17,18,20]

# define the items:
chair = Item(costs_wood=2,costs_metal=1,costs_time_one=3,costs_time_two=1,profit=chair_profit)
table = Item(3,3,3,4,table_profit)
bed = Item(4,4,2,4, bed_profit)
bookcase = Item(5,7,3,3,bookcase_profit)

# Making calculations for one year
months = 12
# specifying available materials
avail_hours_a = 50
avail_hours_b = 30
avail_hours_c = 40
avail_hours_d = 30
avail_wood = [40,59,66,43,50,70,65,80,72,40,34,90]
avail_metal = [24,20,33,40,42,44,45,29,34,38,20,80]

In [ ]:
# Set up the model builder: 
model_builder = ModelBuilder(months=months, avail_wood=avail_wood, avail_metal=avail_metal
                            , avail_hours_a=avail_hours_a,avail_hours_b=avail_hours_b
                            , avail_hours_c=avail_hours_c,avail_hours_d=avail_hours_d
                            , chair=chair,table=table, bed=bed, bookcase=bookcase)
# build the model:
model = model_builder.build_model()


In [ ]:
# Save the model to export a game
#model_builder.save_model('model_v1')

## Random Agent
This agent solves the problem by randomly picking affordable items and producing a random number of the chosen item until nothing more can be produced. 

In [ ]:
# importing a random agent
from agents import RandomAgent

In [ ]:
agents = []
# create the random agent by passing the model
a = RandomAgent(model)
# Step through every month with the strategy of the agent
a.simulate_year()
agents.append(a)

In [ ]:
a.print_profits()

### Plots

In [ ]:
# Plot the profit for each month of the agent compared to the optimal solution
a.plot_profits()

In [ ]:
# Show the amount of produced items per month for the agent
a.plot_produced_items()

In [ ]:
# Show the amount of produced items per month from the optimal solution
a.plot_produced_items(a.optimal_solution)

## ExpensiveMaterialAgent
This agent builds the products that have the highest costs first.
It does not make any differences between the costs, they are all recieve the same weight.

In [ ]:
from agents import ExpensiveMaterialAgent

In [ ]:
e = ExpensiveMaterialAgent(model)
e.simulate_year()
agents.append(e)

In [ ]:
# Compare the agent profit with the optimal solution
e.print_profits()

In [ ]:
e.plot_profits()

In [ ]:
e.plot_produced_items()

In [ ]:
e.plot_produced_items(e.optimal_solution)

## NaiveProfitAgent
This agent builds the products that promise the most profit first.
It does not consider any costs.

In [ ]:
from agents import NaiveProfitAgent
c = NaiveProfitAgent(model)
c.simulate_year()
agents.append(c)

In [ ]:
c.print_profits()

In [ ]:
c.plot_profits()

In [ ]:
c.plot_produced_items()

In [ ]:
c.plot_produced_items(c.optimal_solution)

## MarginProfitAgent
This agent behaves the same as the naive profit agent but considers the costs of the items

In [ ]:
from agents import MarginProfitAgent

In [ ]:
margin_agent = MarginProfitAgent(model)
margin_agent.simulate_year()
agents.append(margin_agent)

In [ ]:
margin_agent.print_profits()

In [ ]:
margin_agent.plot_profits()

In [ ]:
margin_agent.plot_produced_items()

In [ ]:
margin_agent.plot_produced_items(margin_agent.optimal_solution)

## Compare Agents

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

In [ ]:
# Compare the profits of each agent
for a in agents:
    plt.plot(range(1,13), a.profits)
plt.legend([a.name for a in agents])
plt.title('Profits of Every Agent')
plt.xticks(range(1,13))
plt.show()

In [ ]:
sums = [sum(a.profits) for a in agents]
df = pd.DataFrame(sums, index=[a.name for a in agents])
df.columns = ['Profit']
display(df)
df.plot(kind='bar', legend=False,width=0.3,figsize=(7,5),rot=1,title='Profits')
plt.show()

## Review variation between items built
In this section the variation coefficient is used to compare the items built

In [ ]:
from agents import stats
optimal_solution = RandomAgent(model).optimal_solution
variation_between_items = [stats.get_distributed_variation(optimal_solution, m) for m in range(12)]
variation_each_item = [stats.get_month_variation(optimal_solution, k) for k in optimal_solution.produced_items.keys()]

In [ ]:
variation_between_items

In [ ]:
variation_each_item